In [ ]:
# imports
from data_prep import one_hot_decode, get_bars_dataset, one_hot_encode, one_hot_decode_song
import pandas as pd
import numpy as np
from predictions import define_models,predict_sequence,predict_sequence_from_zs,predict_z_and_states
from music21 import converter, instrument, note, chord, stream, midi, environment
%matplotlib inline
import os
from midi_handling import noteArrayToStream, switch_range
from music21 import *
import numpy as np
mdn_dataset_path = "h5_files/mdn_dataset_b1.h5"
vae_weights_path = "weights/512_64/weights-improvement-2455-1.00.hdf5"
import h5py
from numba import jit
from tqdm import tqdm

from ngram_compute import int_list_to_str, compute_ngram_prob
############################ MDN initalisation ###################################
SEQ_LEN = 16
BATCH_SIZE = 64
HIDDEN_UNITS = 512
EPOCHS = 400
SEED = 2345  # set random seed for reproducibility


OUTPUT_DIMENSION = 64
NUMBER_MIXTURES = 10


np.random.seed(SEED)
############################ VAE initalisation ###################################
n_features = 131
timesteps = 16
learning_rate = 0.0001
n_encoder_units = 512
n_decoder_units = n_encoder_units
latent_dim = 64
dropout=0.3
beta=1
############################ MUSESCORE intialisation ###############################
environment.set("musescoreDirectPNGPath", "/usr/bin/musescore")

############################ NGRAM intialisation ###############################
from decimal import *
getcontext().prec = 6 # set max decimal points for ngram
from math import log

Separated in two parts:  
VAE predictions  
MDN predictions  
  
Evaluation is done with 500 16-bar songs from the dataset.  
  
Two real songs are chosen for each evaluation.  
  
For the VAE, each bar is predicted from a corresponding z.  
  
For the MDN, the first z is predicted from the VAE decoder, then the MDN predicts the rest of the song.

In [ ]:
#Choose songs:
"""
song_count = 0
pointer = 0

with h5py.File("h5_files/As_and_Bs.h5", "w") as hpyNgram:

    while song_count < 1000:

        check_song = np.empty((1,16,131))
        songA = np.empty((1,16,131))
        songB = np.empty((1,16,131))

        song_from_fileA = np.load("songs_encoder_inputs/id-" + str(pointer) + ".npy")
        song_lenA = song_from_fileA.shape[0]

        song_from_fileB = np.load("songs_encoder_inputs/id-" + str(pointer+1) + ".npy")
        song_lenB = song_from_fileB.shape[0]

        if (song_lenA >= 10) and (song_lenB >=10):
            print(song_count)
            songA = song_from_fileA
            songB = song_from_fileB

            hpyNgram.create_dataset("A" + str(pointer) + "_z" + str(pointer), data=song_from_fileA)
            hpyNgram.create_dataset("B" + str(pointer) + "_z" + str(pointer+1), data=song_from_fileB)
            song_count +=1
        else:
            print("sometin")

        pointer += 1
"""

# Get VAE

In [ ]:
# define model
train, infenc, infdec = define_models(n_encoder_units=n_encoder_units,
                                        n_decoder_units=n_decoder_units,
                                        latent_dim=latent_dim, 
                                      timesteps=timesteps,
                                      n_features=n_features,
                                      learning_rate=learning_rate,
                                        dropout=dropout,
                                      beta=beta,
                                        epsilon_std=1.)

# Set VAE weights

In [ ]:
train.load_weights(vae_weights_path)
print("Loaded weights from disk")

all_weights = train.get_weights()
decoder_position = len(infenc.get_weights())
decoder_weights = all_weights[decoder_position::]
infdec.set_weights(decoder_weights)

## Initialize empty dataframe

In [ ]:
cols = ['a', 'c0', 'c1', 'c2', 'c3', 'c4', 'b']

df_idx = 0
df1 = pd.DataFrame(columns=cols, index=[df_idx])

for i in range(1):
    df1.loc[i].a = np.nan
    df1.loc[i].c0 = np.nan
    df1.loc[i].c1 = np.nan
    df1.loc[i].c2 = np.nan
    df1.loc[i].c3 = np.nan
    df1.loc[i].c4 = np.nan
    df1.loc[i].b = np.nan
    
with open('ngram_results.csv', 'w') as f:
    df1.to_csv(f, header=True)

## Predict intermediate songs and compute likelihood

Maximum likelihood for a sequence (with Laplace smoothing) is:

```python
MLE = (Count(n grams) + 1)/ (Count(n-1 grams) + V)
```

V is the number of unique tokens in the vocabulary, in this case: 

```python
V = 130
```

The absolute worst MLE for a sequence is -759.3353742710694:

```python
for i in range(16 * 10 - 4):
    prob += log(float(0 + 1 / 0 + 130))
```

In [ ]:
length = 5

z_dataset = h5py.File("h5_files/z_dataset.h5", "r")

with h5py.File("h5_files/As_and_Bs.h5", "r") as A_and_B:
    
    for item in tqdm(list(A_and_B.keys())):      

        # update index for dataframe
        df_idx += 1
        
        cols = ['a', 'c0', 'c1', 'c2', 'c3', 'c4', 'b']

        # create temporary dataframe
        df2 = pd.DataFrame(columns=cols, index=[df_idx])

        
        ################ song A ###################################
        
        # get song A compute ngram MLE
        songA = np.array(A_and_B.get(item))[0:length]
        A = one_hot_decode_song(songA)
        
        # add to dataframe position a
        df2.loc[df_idx].a = compute_ngram_prob(A)

        ################ song B ###################################
        # get B position
        endpos = int(item.split("_")[0][1:]) + 1
        B_pos = "B" + item.split("_")[0][1:] + "_z" +  str( endpos )
    
        # get song B compute ngram MLE
        B = one_hot_decode_song(np.array(A_and_B.get(B_pos))[0:length])
        
        # add to dataframe position b
        df2.loc[df_idx].b = compute_ngram_prob(B)
        
        
        
        
        ################ latent z for interpolation ################
        # get z-position for song 
        z_pos = item.split("_")[1][1:]
        
        # get 5 first z's for songA
        song_zA = np.array(z_dataset.get('z_list' + str(z_pos)))[0:length]
        
        # get 5 first z's for songB
        song_zB = np.array(z_dataset.get('z_list' + str(int(z_pos)+1)))[0:length]

        ################## interpolating ###########################
        alpha = 0.1 
        
        # update alpha 9 times (0.1 - 0.9)
        for i in range(5):
            pred_song = []   
            
            # loop through 5 timesteps of a song
            for j in range(length):
                
                z = alpha*song_zA[j] + float(1-alpha)*song_zB[j]

                encoder_output = predict_sequence_from_zs(infdec,
                                                 z,
                                                 latent_dim,
                                                 seq_len=timesteps,
                                                 temperature=1,
                                                 n_decoder_units=n_decoder_units)
                
                pred_song.append(one_hot_decode(encoder_output.reshape(timesteps, n_features)))

            # compute ngram probability of song and add to dataframe row
            c_pos = "c" + str(i)

            df2.loc[[df_idx],[c_pos]] = compute_ngram_prob(np.ravel(pred_song))
            
            # update alpha
            alpha += 0.2
            
        # append dataframe
        with open('ngram_results.csv', 'a') as f:
            df2.to_csv(f, header=False)


z_dataset.close()